In [1]:
import pandas as pd
filename = "train.csv"
columns = ('click','hour','weekday','useragent', 'slotprice', 'region', 'city', 'slotwidth', 'slotheight', 'advertiser', 'usertag')
df = pd.read_csv(filename,usecols=columns)


In [2]:
df.insert(1,'browser',df['useragent'])
browsers = []
for name in df["useragent"].map(lambda x:x.split('_')[1:][0]):
    if name not in browsers:
        browsers.append(name)
data_bro = []
for bro in df["useragent"].map(lambda x:x.split('_')[1:][0]):
    data_bro.append(browsers.index(bro))
df['browser'] = data_bro
    
df.insert(1,'os',df['useragent'])

oss = []
for name in df["useragent"].map(lambda x:x.split('_')[0]):
    if name not in oss:
        oss.append(name)
data_os = []
for os in df["useragent"].map(lambda x:x.split('_')[0]):
    data_os.append(oss.index(os))
df['os'] = data_os

df.insert(1,'size',df['slotwidth'])
width = df['slotwidth'].astype('str')
height = df['slotheight'].astype('str')
size = width.str.cat(height, sep='*')
df['size']=size
del df['slotwidth']
del df['slotheight']

df.insert(1,'floorprice',df['slotprice'])
floor = []
for i in df["slotprice"]:
    if i in range(1,10):
        floor.append(1)
    elif i in range(11,50):
        floor.append(2)
    elif i in range(51,100):
        floor.append(3)
    elif i == 0:
        floor.append(0)
    else:
        floor.append(4)
df['floorprice'] = floor 

In [3]:
col = ['slotprice','size', 'os', 'browser']
for column in col: 
    df[column]= pd.factorize(df[column].values , sort=True)[0] + 1 
df['advertiser_n'] = pd.factorize(df['advertiser'].values , sort=True)[0] + 1 

In [4]:
import collections
users = collections.defaultdict()
i = 0
for user in df["usertag"]:
    users[i] = str(user).split(',')
    i += 1
s = pd.Series(users)
usrtag = pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)
del df["usertag"]

In [6]:
# dummy_price = pd.get_dummies(df['floorprice'], prefix='floorprice')  
# dummy_os = pd.get_dummies(df['os'], prefix='os')  
# dummy_browser = pd.get_dummies(df['browser'], prefix='browser')  
# dummy_weekday = pd.get_dummies(df['weekday'], prefix='weekday') 
# dummy_hour = pd.get_dummies(df['hour'], prefix='hour') 
# # cols_to_keep = ['click'] 
# data_train = df.join(dummy_price)
# data_train = data_train.join(dummy_os)
# data_train = data_train.join(dummy_browser)
# data_train = data_train.join(dummy_weekday)
# data_train = data_train.join(dummy_hour)
# data_train = data_train.join(usrtag)

In [5]:

data_train = df.join(usrtag)

In [6]:
del data_train['useragent']
del data_train['nan']

In [7]:
import numpy as np
np.random.seed(10)

import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline

In [9]:
X_train = data_train[data_train.columns[1:]]
y_train = pd.DataFrame(data_train[data_train.columns[0]])
X_train_lr = data_train[data_train.columns[1:]]
y_train_lr = pd.DataFrame(data_train[data_train.columns[0]])

In [8]:
train_advertisers = {}
for name, group in data_train.groupby('advertiser'):
    train_advertisers[name] = pd.DataFrame(group)

In [9]:
adv_train_1458 = train_advertisers[1458]
y_train_1458 = train_advertisers[1458]['click']
adv_train_2259 = train_advertisers[2259]
y_train_2259 = train_advertisers[2259]['click']
adv_train_2261 = train_advertisers[2261]
y_train_2261 = train_advertisers[2261]['click']
adv_train_2821 = train_advertisers[2821]
y_train_2821 = train_advertisers[2821]['click']
adv_train_2997 = train_advertisers[2997]
y_train_2997 = train_advertisers[2997]['click']
adv_train_3358 = train_advertisers[3358]
y_train_3358 = train_advertisers[3358]['click']
adv_train_3386 = train_advertisers[3386]
y_train_3386 = train_advertisers[3386]['click']
adv_train_3427 = train_advertisers[3427]
y_train_3427 = train_advertisers[3427]['click']
adv_train_3476 = train_advertisers[3476]
y_train_3476 = train_advertisers[3476]['click']

In [10]:
del adv_train_1458['advertiser_n']
del adv_train_1458['click']
del adv_train_2259['advertiser_n']
del adv_train_2259['click']
del adv_train_2261['advertiser_n']
del adv_train_2261['click']
del adv_train_2821['advertiser_n']
del adv_train_2821['click']
del adv_train_2997['advertiser_n']
del adv_train_2997['click']
del adv_train_3358['advertiser_n']
del adv_train_3358['click']
del adv_train_3386['advertiser_n']
del adv_train_3386['click']
del adv_train_3427['advertiser_n']
del adv_train_3427['click']
del adv_train_3476['advertiser_n']
del adv_train_3476['click']

In [13]:
del X_train['advertiser']
del X_train_lr['advertiser']

In [11]:
import pandas as pd
filename = "validation.csv"
columns = ('click','hour','weekday','useragent', 'slotprice', 'region', 'city', 'slotwidth', 'slotheight', 'advertiser', 'usertag')
df_val = pd.read_csv(filename,usecols=columns)
df_val.insert(1,'browser',df_val['useragent'])
browsers = []
for name in df_val["useragent"].map(lambda x:x.split('_')[1:][0]):
    if name not in browsers:
        browsers.append(name)
data_bro = []
for bro in df_val["useragent"].map(lambda x:x.split('_')[1:][0]):
    data_bro.append(browsers.index(bro))
df_val['browser'] = data_bro
    
df_val.insert(1,'os',df_val['useragent'])

oss = []
for name in df_val["useragent"].map(lambda x:x.split('_')[0]):
    if name not in oss:
        oss.append(name)
data_os = []
for os in df_val["useragent"].map(lambda x:x.split('_')[0]):
    data_os.append(oss.index(os))
df_val['os'] = data_os

df_val.insert(1,'size',df_val['slotwidth'])
width = df_val['slotwidth'].astype('str')
height = df_val['slotheight'].astype('str')
size = width.str.cat(height, sep='*')
df_val['size']=size


del df_val['slotwidth']
del df_val['slotheight']

df_val.insert(1,'floorprice',df_val['slotprice'])
floor = []
for i in df_val["slotprice"]:
    if i in range(1,10):
        floor.append(1)
    elif i in range(11,50):
        floor.append(2)
    elif i in range(51,100):
        floor.append(3)
    elif i == 0:
        floor.append(0)
    else:
        floor.append(4)
df_val['floorprice'] = floor 
col = ['slotprice','size', 'os', 'browser']
for column in col: 
    df_val[column]= pd.factorize(df_val[column].values , sort=True)[0] + 1
df_val['advertiser_n']= pd.factorize(df_val['advertiser'].values , sort=True)[0] + 1


import collections
users = collections.defaultdict()
i = 0
for user in df_val["usertag"]:
    users[i] = str(user).split(',')
    i += 1
s = pd.Series(users)
usrtag = pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)
del df_val["usertag"]

# dummy_price = pd.get_dummies(df_val['floorprice'], prefix='floorprice')  
# dummy_os = pd.get_dummies(df_val['os'], prefix='os')  
# dummy_browser = pd.get_dummies(df_val['browser'], prefix='browser')  
# dummy_weekday = pd.get_dummies(df_val['weekday'], prefix='weekday') 
# dummy_hour = pd.get_dummies(df_val['hour'], prefix='hour') 
# # cols_to_keep = ['click'] 
# data_val = df_val.join(dummy_price)
# data_val = data_val.join(dummy_os)
# data_val = data_val.join(dummy_browser)
# data_val = data_val.join(dummy_weekday)
# data_val = data_val.join(dummy_hour)
# data_val = data_val.join(usrtag)


data_val = df_val.join(usrtag)

In [12]:
del data_val['nan']
del data_val['useragent']

In [16]:
X_val = data_val[data_val.columns[1:]]
y_val = pd.DataFrame(data_val[data_val.columns[0]])
X_val_lr = data_val[data_val.columns[1:]]
y_val_lr = pd.DataFrame(data_val[data_val.columns[0]])

In [13]:
val_advertisers = {}
for name, group in data_val.groupby('advertiser'):
    val_advertisers[name] = pd.DataFrame(group)


In [14]:
adv_val_1458 = val_advertisers[1458]
y_val_1458 = val_advertisers[1458]['click']
adv_val_2259 = val_advertisers[2259]
y_val_2259 = val_advertisers[2259]['click']
adv_val_2261 = val_advertisers[2261]
y_val_2261 = val_advertisers[2261]['click']
adv_val_2821 = val_advertisers[2821]
y_val_2821 = val_advertisers[2821]['click']
adv_val_2997 = val_advertisers[2997]
y_val_2997 = val_advertisers[2997]['click']
adv_val_3358 = val_advertisers[3358]
y_val_3358 = val_advertisers[3358]['click']
adv_val_3386 = val_advertisers[3386]
y_val_3386 = val_advertisers[3386]['click']
adv_val_3427 = val_advertisers[3427]
y_val_3427 = val_advertisers[3427]['click']
adv_val_3476 = val_advertisers[3476]
y_val_3476 = val_advertisers[3476]['click']

In [15]:
del adv_val_1458['advertiser_n']
del adv_val_1458['click']
del adv_val_2259['advertiser_n']
del adv_val_2259['click']
del adv_val_2261['advertiser_n']
del adv_val_2261['click']
del adv_val_2821['advertiser_n']
del adv_val_2821['click']
del adv_val_2997['advertiser_n']
del adv_val_2997['click']
del adv_val_3358['advertiser_n']
del adv_val_3358['click']
del adv_val_3386['advertiser_n']
del adv_val_3386['click']
del adv_val_3427['advertiser_n']
del adv_val_3427['click']
del adv_val_3476['advertiser_n']
del adv_val_3476['click']

In [20]:
del X_val['advertiser']
del X_val_lr['advertiser']

In [16]:
y_val_1458 = pd.DataFrame(y_val_1458)
y_val_2259 = pd.DataFrame(y_val_2259)
y_val_2261 = pd.DataFrame(y_val_2261)
y_val_2821 = pd.DataFrame(y_val_2821)
y_val_2997 = pd.DataFrame(y_val_2997)
y_val_3358 = pd.DataFrame(y_val_3358)
y_val_3386 = pd.DataFrame(y_val_3386)
y_val_3427 = pd.DataFrame(y_val_3427)
y_val_3476 = pd.DataFrame(y_val_3476)

y_train_1458 = pd.DataFrame(y_train_1458)
y_train_2259 = pd.DataFrame(y_train_2259)
y_train_2261 = pd.DataFrame(y_train_2261)
y_train_2821 = pd.DataFrame(y_train_2821)
y_train_2997 = pd.DataFrame(y_train_2997)
y_train_3358 = pd.DataFrame(y_train_3358)
y_train_3386 = pd.DataFrame(y_train_3386)
y_train_3427 = pd.DataFrame(y_train_3427)
y_train_3476 = pd.DataFrame(y_train_3476)

In [17]:
y_val_1458.to_csv('y_val_1458.csv')
y_val_2259.to_csv('y_val_2259.csv')
y_val_2261.to_csv('y_val_2261.csv')
y_val_2821.to_csv('y_val_2821.csv')
y_val_2997.to_csv('y_val_2997.csv')
y_val_3358.to_csv('y_val_3358.csv')
y_val_3386.to_csv('y_val_3386.csv')
y_val_3427.to_csv('y_val_3427.csv')
y_val_3476.to_csv('y_val_3476.csv')

y_train_1458.to_csv('y_train_1458.csv')
y_train_2259.to_csv('y_train_2259.csv')
y_train_2261.to_csv('y_train_2261.csv')
y_train_2821.to_csv('y_train_2821.csv')
y_train_2997.to_csv('y_train_2997.csv')
y_train_3358.to_csv('y_train_3358.csv')
y_train_3386.to_csv('y_train_3386.csv')
y_train_3427.to_csv('y_train_3427.csv')
y_train_3476.to_csv('y_train_3476.csv')
y_train_3358.to_csv('y_train_3358.csv')

In [18]:
adv_val_1458.to_csv('adv_val_1458.csv')
adv_val_2259.to_csv('adv_val_2259.csv')
adv_val_2261.to_csv('adv_val_2261.csv')
adv_val_2821.to_csv('adv_val_2821.csv')
adv_val_2997.to_csv('adv_val_2997.csv')
adv_val_3358.to_csv('adv_val_3358.csv')
adv_val_3386.to_csv('adv_val_3386.csv')
adv_val_3427.to_csv('adv_val_3427.csv')
adv_val_3476.to_csv('adv_val_3476.csv')


In [19]:
adv_train_1458.to_csv('adv_train_1458.csv')
adv_train_2259.to_csv('adv_train_2259.csv')
adv_train_2261.to_csv('adv_train_2261.csv')
adv_train_2821.to_csv('adv_train_2821.csv')
adv_train_2997.to_csv('adv_train_2997.csv')
adv_train_3358.to_csv('adv_train_3358.csv')
adv_train_3427.to_csv('adv_train_3427.csv')
adv_train_3476.to_csv('adv_train_3476.csv')
adv_train_3386.to_csv('adv_train_3386.csv')



In [96]:
X_val.to_csv('X_val.csv')
X_val_lr.to_csv('X_val_lr.csv')
X_train.to_csv('X_train.csv')
X_train_lr.to_csv('X_train_lr.csv')

In [201]:
y_val = pd.DataFrame(y_val)
y_val_lr = pd.DataFrame(y_val_lr)
y_train = pd.DataFrame(y_train)
y_train_lr = pd.DataFrame(y_train_lr)

In [202]:
y_val.to_csv('y_val.csv')
y_val_lr.to_csv('y_val_lr.csv')
y_train.to_csv('y_train.csv')
y_train_lr.to_csv('y_train_lr.csv')

In [1]:
import pandas as pd
filename = "adv_train_1458.csv"
adv_train_1458 = pd.read_csv(filename)

filename = "y_train_1458.csv"
y_train_1458 = pd.read_csv(filename)

filename = "adv_val_1458.csv"
adv_val_1458 = pd.read_csv(filename)

filename = "y_val_1458.csv"
y_val_1458 = pd.read_csv(filename)

In [2]:
del adv_train_1458['Unnamed: 0']
del y_val_1458['Unnamed: 0']
del adv_val_1458['Unnamed: 0']
del y_train_1458['Unnamed: 0']

In [3]:
from scipy.sparse.construct import hstack
from sklearn.model_selection import train_test_split
from sklearn.datasets.svmlight_format import load_svmlight_file
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score
from sklearn.preprocessing.data import OneHotEncoder
import numpy as np

gbdt_1458 = GradientBoostingClassifier(n_estimators=40, max_depth=3, verbose=0,max_features=0.5)

gbdt_1458.fit(adv_train_1458, y_train_1458)
y_pred_gbdt1458 = gbdt_1458.predict_proba(adv_val_1458)[:, 1]
gbdt_auc1458 = roc_auc_score(y_val_1458, y_pred_gbdt1458)
print('gbdt auc 1458: %.5f' % gbdt_auc1458)



/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


gbdt auc 1458: 0.95095


In [4]:
lr = LogisticRegression()
lr.fit(adv_train_1458, y_train_1458)    
y_pred_test = lr.predict_proba(adv_val_1458)[:, 1]
lr_test_auc = roc_auc_score(y_val_1458, y_pred_test)
print('LR AUC 1458: %.5f' % lr_test_auc)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LR AUC 1458: 0.82172


In [4]:
X_train_leaves = gbdt_1458.apply(adv_train_1458)[:,:,0]
X_test_leaves = gbdt_1458.apply(adv_val_1458)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbdtenc = OneHotEncoder()
X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))



In [6]:
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train_1458)

y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
gbdt_lr_auc1 = roc_auc_score(y_val_1458, y_pred_gbdtlr1)
print('GBDT+LR AUC 1458: %.5f' % gbdt_lr_auc1)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


基于GBDT特征编码后的LR AUC: 0.95577


In [1]:
import pandas as pd
filename = "adv_train_2259.csv"
adv_train_2259 = pd.read_csv(filename)

filename = "y_train_2259.csv"
y_train_2259 = pd.read_csv(filename)

filename = "adv_val_2259.csv"
adv_val_2259 = pd.read_csv(filename)

filename = "y_val_2259.csv"
y_val_2259 = pd.read_csv(filename)

In [2]:
del adv_train_2259['Unnamed: 0']
del y_val_2259['Unnamed: 0']
del adv_val_2259['Unnamed: 0']
del y_train_2259['Unnamed: 0']

In [3]:
from scipy.sparse.construct import hstack
from sklearn.model_selection import train_test_split
from sklearn.datasets.svmlight_format import load_svmlight_file
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.metrics.ranking import roc_auc_score
from sklearn.preprocessing.data import OneHotEncoder
import numpy as np

gbdt_2259 = GradientBoostingClassifier(n_estimators=40, max_depth=3, verbose=0,max_features=0.5)

gbdt_2259.fit(adv_train_2259, y_train_2259)
y_pred_gbdt2259 = gbdt_2259.predict(adv_val_2259)
gbdt_auc2259 = roc_auc_score(y_val_2259, y_pred_gbdt2259)
print('gbdt auc 2259: %.5f' % gbdt_auc2259)




/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


gbdt auc 2259: 0.50407


In [4]:
from sklearn.metrics.ranking import roc_auc_score
from sklearn.linear_model.logistic import LogisticRegression
lr = LogisticRegression()
lr.fit(adv_train_2259, y_train_2259)    
y_pred_test = lr.predict(adv_val_2259)
lr_test_auc = roc_auc_score(y_val_2259, y_pred_test)
print('LR AUC 2259: %.5f' % lr_test_auc)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LR AUC 2259: 0.50000


In [4]:
X_train_leaves = gbdt_2259.apply(adv_train_2259)[:,:,0]
X_test_leaves = gbdt_2259.apply(adv_val_2259)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbdtenc = OneHotEncoder()
X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))




In [5]:
from sklearn.metrics.ranking import roc_auc_score
from sklearn.linear_model.logistic import LogisticRegression
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train_2259)

y_pred_gbdtlr1 = lr.predict(X_trans[train_rows:, :])
gbdt_lr_auc1 = roc_auc_score(y_val_2259, y_pred_gbdtlr1)
print('GBDT+LR AUC 2259: %.5f' % gbdt_lr_auc1)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBDT+LR AUC 2259: 0.39338


In [1]:
import pandas as pd
filename = "adv_train_2261.csv"
adv_train_2261 = pd.read_csv(filename)

filename = "y_train_2261.csv"
y_train_2261 = pd.read_csv(filename)

filename = "adv_val_2261.csv"
adv_val_2261 = pd.read_csv(filename)

filename = "y_val_2261.csv"
y_val_2261 = pd.read_csv(filename)

In [2]:
del adv_train_2261['Unnamed: 0']
del y_val_2261['Unnamed: 0']
del adv_val_2261['Unnamed: 0']
del y_train_2261['Unnamed: 0']

In [3]:
from scipy.sparse.construct import hstack
from sklearn.model_selection import train_test_split
from sklearn.datasets.svmlight_format import load_svmlight_file
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score
from sklearn.preprocessing.data import OneHotEncoder
import numpy as np

gbdt_2261 = GradientBoostingClassifier(n_estimators=40, max_depth=3, verbose=0,max_features=0.5)

gbdt_2261.fit(adv_train_2261, y_train_2261)
y_pred_gbdt2261 = gbdt_2261.predict_proba(adv_val_2261)[:, 1]
gbdt_auc2261 = roc_auc_score(y_val_2261, y_pred_gbdt2261)
print('gbdt auc 2261: %.5f' % gbdt_auc2261)





/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


gbdt auc 2261: 0.66647


In [6]:

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score

lr = LogisticRegression()
lr.fit(adv_train_2261, y_train_2261)    
y_pred_test = lr.predict_proba(adv_val_2261)[:, 1]
lr_test_auc = roc_auc_score(y_val_2261, y_pred_test)
print('LR AUC 2261: %.5f' % lr_test_auc)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LR AUC 2261: 0.58552


In [4]:
X_train_leaves = gbdt_2261.apply(adv_train_2261)[:,:,0]
X_test_leaves = gbdt_2261.apply(adv_val_2261)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbdtenc = OneHotEncoder()
X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))


In [5]:
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train_2261)

y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
gbdt_lr_auc1 = roc_auc_score(y_val_2261, y_pred_gbdtlr1)
print('GBDT+LR AUC 2261: %.5f' % gbdt_lr_auc1)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBDT+LR AUC 2261: 0.87845


In [1]:
import pandas as pd
filename = "adv_train_2821.csv"
adv_train_2821 = pd.read_csv(filename)

filename = "y_train_2821.csv"
y_train_2821 = pd.read_csv(filename)

filename = "adv_val_2821.csv"
adv_val_2821 = pd.read_csv(filename)

filename = "y_val_2821.csv"
y_val_2821 = pd.read_csv(filename)

In [2]:
del adv_train_2821['Unnamed: 0']
del y_val_2821['Unnamed: 0']
del adv_val_2821['Unnamed: 0']
del y_train_2821['Unnamed: 0']

In [3]:
from scipy.sparse.construct import hstack
from sklearn.model_selection import train_test_split
from sklearn.datasets.svmlight_format import load_svmlight_file
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score
from sklearn.preprocessing.data import OneHotEncoder
import numpy as np

gbdt_2821 = GradientBoostingClassifier(n_estimators=40, max_depth=3, verbose=0,max_features=0.5)

gbdt_2821.fit(adv_train_2821, y_train_2821)
y_pred_gbdt2821 = gbdt_2821.predict_proba(adv_val_2821)[:, 1]
gbdt_auc2821 = roc_auc_score(y_val_2821, y_pred_gbdt2821)
print('gbdt auc 2821: %.5f' % gbdt_auc2821)






/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


gbdt auc 2821: 0.54296


In [3]:

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score

lr = LogisticRegression()
lr.fit(adv_train_2821, y_train_2821)    
y_pred_test = lr.predict_proba(adv_val_2821)[:, 1]
lr_test_auc = roc_auc_score(y_val_2821, y_pred_test)
print('LR AUC 2821: %.5f' % lr_test_auc)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LR AUC 2821: 0.50167


In [5]:
X_train_leaves = gbdt_2821.apply(adv_train_2821)[:,:,0]
X_test_leaves = gbdt_2821.apply(adv_val_2821)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbdtenc = OneHotEncoder()
X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))



In [6]:
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train_2821)

y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
gbdt_lr_auc1 = roc_auc_score(y_val_2821, y_pred_gbdtlr1)
print('GBDT+LR AUC 2821: %.5f' % gbdt_lr_auc1)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBDT+LR AUC 2821: 0.56635


In [1]:
import pandas as pd
filename = "adv_train_2997.csv"
adv_train_2997 = pd.read_csv(filename)

filename = "y_train_2997.csv"
y_train_2997 = pd.read_csv(filename)

filename = "adv_val_2997.csv"
adv_val_2997 = pd.read_csv(filename)

filename = "y_val_2997.csv"
y_val_2997 = pd.read_csv(filename)

In [2]:
del adv_train_2997['Unnamed: 0']
del y_val_2997['Unnamed: 0']
del adv_val_2997['Unnamed: 0']
del y_train_2997['Unnamed: 0']

In [3]:
from scipy.sparse.construct import hstack
from sklearn.model_selection import train_test_split
from sklearn.datasets.svmlight_format import load_svmlight_file
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score
from sklearn.preprocessing.data import OneHotEncoder
import numpy as np

gbdt_2997 = GradientBoostingClassifier(n_estimators=40, max_depth=3, verbose=0,max_features=0.5)

gbdt_2997.fit(adv_train_2997, y_train_2997)
y_pred_gbdt2997 = gbdt_2997.predict_proba(adv_val_2997)[:, 1]
gbdt_auc2997 = roc_auc_score(y_val_2997, y_pred_gbdt2997)
print('gbdt auc 2997: %.5f' % gbdt_auc2997)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


gbdt auc 2997: 0.60564


In [4]:

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score

lr = LogisticRegression()
lr.fit(adv_train_2997, y_train_2997)    
y_pred_test = lr.predict_proba(adv_val_2997)[:, 1]
lr_test_auc = roc_auc_score(y_val_2997, y_pred_test)
print('LR AUC 2997: %.5f' % lr_test_auc)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LR AUC 2997: 0.70655


In [4]:
X_train_leaves = gbdt_2997.apply(adv_train_2997)[:,:,0]
X_test_leaves = gbdt_2997.apply(adv_val_2997)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbdtenc = OneHotEncoder()
X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))


In [5]:
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train_2997)

y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
gbdt_lr_auc1 = roc_auc_score(y_val_2997, y_pred_gbdtlr1)
print('GBDT+LR AUC 2997: %.5f' % gbdt_lr_auc1)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBDT+LR AUC 2997: 0.61134


In [1]:
import pandas as pd
filename = "adv_train_3358.csv"
adv_train_3358 = pd.read_csv(filename)

filename = "y_train_3358.csv"
y_train_3358 = pd.read_csv(filename)

filename = "adv_val_3358.csv"
adv_val_3358 = pd.read_csv(filename)

filename = "y_val_3358.csv"
y_val_3358 = pd.read_csv(filename)

In [2]:
del adv_train_3358['Unnamed: 0']
del y_val_3358['Unnamed: 0']
del adv_val_3358['Unnamed: 0']
del y_train_3358['Unnamed: 0']

In [3]:
from scipy.sparse.construct import hstack
from sklearn.model_selection import train_test_split
from sklearn.datasets.svmlight_format import load_svmlight_file
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score
from sklearn.preprocessing.data import OneHotEncoder
import numpy as np

gbdt_3358 = GradientBoostingClassifier(n_estimators=40, max_depth=3, verbose=0,max_features=0.5)

gbdt_3358.fit(adv_train_3358, y_train_3358)
y_pred_gbdt3358 = gbdt_3358.predict_proba(adv_val_3358)[:, 1]
gbdt_auc3358 = roc_auc_score(y_val_3358, y_pred_gbdt3358)
print('gbdt auc 3358: %.5f' % gbdt_auc3358)


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


gbdt auc 3358: 0.89625


In [6]:

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score

lr = LogisticRegression()
lr.fit(adv_train_3358, y_train_3358)    # 预测及AUC评测
y_pred_test = lr.predict_proba(adv_val_3358)[:, 1]
lr_test_auc = roc_auc_score(y_val_3358, y_pred_test)
print('LR AUC 3358: %.5f' % lr_test_auc)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LR AUC 3358: 0.94068


In [4]:
X_train_leaves = gbdt_3358.apply(adv_train_3358)[:,:,0]
X_test_leaves = gbdt_3358.apply(adv_val_3358)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbdtenc = OneHotEncoder()
X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))


In [5]:
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train_3358)

y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
gbdt_lr_auc1 = roc_auc_score(y_val_3358, y_pred_gbdtlr1)
print('GBDT+LR AUC 3358: %.5f' % gbdt_lr_auc1)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBDT+LR AUC 3358: 0.91249


In [1]:
import pandas as pd
filename = "adv_train_3427.csv"
adv_train_3427 = pd.read_csv(filename)

filename = "y_train_3427.csv"
y_train_3427 = pd.read_csv(filename)

filename = "adv_val_3427.csv"
adv_val_3427 = pd.read_csv(filename)

filename = "y_val_3427.csv"
y_val_3427 = pd.read_csv(filename)

In [2]:
del adv_train_3427['Unnamed: 0']
del y_val_3427['Unnamed: 0']
del adv_val_3427['Unnamed: 0']
del y_train_3427['Unnamed: 0']

In [3]:
from scipy.sparse.construct import hstack
from sklearn.model_selection import train_test_split
from sklearn.datasets.svmlight_format import load_svmlight_file
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score
from sklearn.preprocessing.data import OneHotEncoder
import numpy as np

gbdt_3427 = GradientBoostingClassifier(n_estimators=40, max_depth=3, verbose=0,max_features=0.5)

gbdt_3427.fit(adv_train_3427, y_train_3427)
y_pred_gbdt3427 = gbdt_3427.predict_proba(adv_val_3427)[:, 1]
gbdt_auc3427 = roc_auc_score(y_val_3427, y_pred_gbdt3427)
print('gbdt auc 3427: %.5f' % gbdt_auc3427)



/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


gbdt auc 3427: 0.84665


In [6]:

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score

lr = LogisticRegression()
lr.fit(adv_train_3427, y_train_3427)    # 预测及AUC评测
y_pred_test = lr.predict_proba(adv_val_3427)[:, 1]
lr_test_auc = roc_auc_score(y_val_3427, y_pred_test)
print('LR AUC 3427: %.5f' % lr_test_auc)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LR AUC 3427: 0.86618


In [4]:
X_train_leaves = gbdt_3427.apply(adv_train_3427)[:,:,0]
X_test_leaves = gbdt_3427.apply(adv_val_3427)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbdtenc = OneHotEncoder()
X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))



In [5]:
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train_3427)

y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
gbdt_lr_auc1 = roc_auc_score(y_val_3427, y_pred_gbdtlr1)
print('GBDT+LR AUC 3427: %.5f' % gbdt_lr_auc1)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBDT+LR AUC 3427: 0.92116


In [1]:
import pandas as pd
filename = "adv_train_3476.csv"
adv_train_3476 = pd.read_csv(filename)

filename = "y_train_3476.csv"
y_train_3476 = pd.read_csv(filename)

filename = "adv_val_3476.csv"
adv_val_3476 = pd.read_csv(filename)

filename = "y_val_3476.csv"
y_val_3476 = pd.read_csv(filename)

In [2]:
del adv_train_3476['Unnamed: 0']
del y_val_3476['Unnamed: 0']
del adv_val_3476['Unnamed: 0']
del y_train_3476['Unnamed: 0']

In [3]:
from scipy.sparse.construct import hstack
from sklearn.model_selection import train_test_split
from sklearn.datasets.svmlight_format import load_svmlight_file
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score
from sklearn.preprocessing.data import OneHotEncoder
import numpy as np

gbdt_3476 = GradientBoostingClassifier(n_estimators=40, max_depth=3, verbose=0,max_features=0.5)

gbdt_3476.fit(adv_train_3476, y_train_3476)
y_pred_gbdt3476 = gbdt_3476.predict_proba(adv_val_3476)[:, 1]
gbdt_auc3476 = roc_auc_score(y_val_3476, y_pred_gbdt3476)
print('gbdt auc 3476: %.5f' % gbdt_auc3476)


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


gbdt auc 3476: 0.83989


In [6]:

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score

lr = LogisticRegression()
lr.fit(adv_train_3476, y_train_3476)    
y_pred_test = lr.predict_proba(adv_val_3476)[:, 1]
lr_test_auc = roc_auc_score(y_val_3476, y_pred_test)
print('LR AUC 3476: %.5f' % lr_test_auc)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LR AUC 3476: 0.74270


In [4]:
X_train_leaves = gbdt_3476.apply(adv_train_3476)[:,:,0]
X_test_leaves = gbdt_3476.apply(adv_val_3476)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbdtenc = OneHotEncoder()
X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))




In [5]:
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train_3476)

y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
gbdt_lr_auc1 = roc_auc_score(y_val_3476, y_pred_gbdtlr1)
print('GBDT+LR AUC 3476: %.5f' % gbdt_lr_auc1)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBDT+LR AUC 3476: 0.86027


In [20]:
import pandas as pd
filename = "adv_train_3386.csv"
adv_train_3386 = pd.read_csv(filename)

filename = "y_train_3386.csv"
y_train_3386 = pd.read_csv(filename)

filename = "adv_val_3386.csv"
adv_val_3386 = pd.read_csv(filename)

filename = "y_val_3386.csv"
y_val_3386 = pd.read_csv(filename)

In [21]:
del adv_train_3386['Unnamed: 0']
del y_val_3386['Unnamed: 0']
del adv_val_3386['Unnamed: 0']
del y_train_3386['Unnamed: 0']

In [22]:
from scipy.sparse.construct import hstack
from sklearn.model_selection import train_test_split
from sklearn.datasets.svmlight_format import load_svmlight_file
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score
from sklearn.preprocessing.data import OneHotEncoder
import numpy as np

gbdt_3386 = GradientBoostingClassifier(n_estimators=40, max_depth=3, verbose=0,max_features=0.5)

gbdt_3386.fit(adv_train_3386, y_train_3386)
y_pred_gbdt3386 = gbdt_3386.predict_proba(adv_val_3386)[:, 1]
gbdt_auc3386 = roc_auc_score(y_val_3386, y_pred_gbdt3386)
print('gbdt auc 3386: %.5f' % gbdt_auc3386)



/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


gbdt auc 3386: 0.73163


In [26]:

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score

lr = LogisticRegression()
lr.fit(adv_train_3386, y_train_3386)    
y_pred_test = lr.predict_proba(adv_val_3386)[:, 1]
lr_test_auc = roc_auc_score(y_val_3386, y_pred_test)
print('LR AUC 3386: %.5f' % lr_test_auc)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LR AUC 3386: 0.61763


In [23]:
X_train_leaves = gbdt_3386.apply(adv_train_3386)[:,:,0]
X_test_leaves = gbdt_3386.apply(adv_val_3386)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbdtenc = OneHotEncoder()
X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))

In [24]:
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train_3386)

y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
gbdt_lr_auc1 = roc_auc_score(y_val_3386, y_pred_gbdtlr1)
print('GBDT+LR AUC 3386: %.5f' % gbdt_lr_auc1)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBDT+LR AUC 3386: 0.74590


In [4]:
import pandas as pd
filename = "X_train.csv"
X_train = pd.read_csv(filename)

filename = "y_train.csv"
y_train = pd.read_csv(filename)

filename = "X_val.csv"
X_val = pd.read_csv(filename)

filename = "y_val.csv"
y_val = pd.read_csv(filename)

In [5]:
del X_train['Unnamed: 0']
del y_train['Unnamed: 0']
del X_val['Unnamed: 0']
del y_val['Unnamed: 0']

In [6]:
from scipy.sparse.construct import hstack
from sklearn.model_selection import train_test_split
from sklearn.datasets.svmlight_format import load_svmlight_file
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score
from sklearn.preprocessing.data import OneHotEncoder
import numpy as np

gbdt = GradientBoostingClassifier(n_estimators=40, max_depth=3, verbose=0,max_features=0.5)
gbdt.fit(X_train, y_train)
y_pred_gbdt = gbdt.predict_proba(np.array(X_val))[:, 1]
gbdt_auc = roc_auc_score(y_val, y_pred_gbdt)
print('gbdt auc total: %.5f' % gbdt_auc)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


gbdt auc total: 0.82685


In [9]:

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score
lr = LogisticRegression()
lr.fit(X_train, y_train)    # 预测及AUC评测
y_pred_test = lr.predict_proba(X_val)[:, 1]
lr_test_auc = roc_auc_score(y_val, y_pred_test)
print('LR AUC: %.5f' % lr_test_auc)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LR AUC: 0.79856


In [12]:
X_train_leaves = gbdt.apply(X_train)[:,:,0]
X_test_leaves = gbdt.apply(X_val)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbdtenc = OneHotEncoder()
X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))


In [13]:
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train)

y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
gbdt_lr_auc1 = roc_auc_score(y_val, y_pred_gbdtlr1)
print('GBDT+LR AUC total: %.5f' % gbdt_lr_auc1)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBDT+LR AUC total: 0.85211


In [30]:
# lr.fit(X_train_ext, y_train)
# y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
# gbdt_lr_auc2 = roc_auc_score(y_val, y_pred_gbdtlr2)
# print('基于组合特征的LR AUC: %.5f' % gbdt_lr_auc2)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


基于组合特征的LR AUC: 0.84427


In [ ]:
print(y_pred_gbdtlr1)

In [ ]:
'CPM':df.groupby('advertiser')['payprice'].sum()*1000/(df.groupby('advertiser')['advertiser'].agg('count')*1000)
'eCPC':df.groupby('advertiser')['payprice'].sum()/(df.groupby('advertiser')['click'].sum()*1000)

In [8]:
import pandas as pd
filename = "validation.csv"
data_val = pd.read_csv(filename)
df_val = pd.DataFrame(data_val)

import pandas as pd
filename = "train.csv"
data_train = pd.read_csv(filename)
df_train = pd.DataFrame(data_train)

In [12]:
avgCTR = df_train['click'].sum()/df_train['click'].agg('count')
base = df_train['payprice'].sum()/df_train['payprice'].agg('count')
for i in range(-50,70):
    sum = 6250 * 1000
    clicks = 0
    payprices = 0
    num = 0
    for pay,click,predict in zip(df_val["payprice"],df_val['click'],y_pred_test):
        sum_sub = sum - pay
        bidprice = (base * predict/avgCTR)-i
        if bidprice-pay > 0 and sum_sub >= 0:
            sum = sum_sub
            clicks += click
            payprices += pay
            num += 1
    print(i, clicks, sum, payprices, num)

-50 85 0 6250000 119472
-49 86 0 6250000 119921
-48 87 0 6250000 120365
-47 90 0 6250000 120826
-46 91 0 6250000 121308
-45 93 0 6250000 121773
-44 93 0 6250000 122281
-43 95 0 6250000 122731
-42 96 0 6250000 123205
-41 97 0 6250000 123767
-40 98 0 6250000 124263
-39 98 0 6250000 124792
-38 99 0 6250000 125242
-37 99 0 6250000 125762
-36 99 0 6250000 126337
-35 99 0 6250000 126902
-34 102 0 6250000 127441
-33 103 0 6250000 127936
-32 105 0 6250000 128523
-31 106 0 6250000 129100
-30 106 0 6250000 129580
-29 107 0 6250000 130120
-28 109 0 6250000 130703
-27 110 0 6250000 131185
-26 113 0 6250000 131820
-25 114 0 6250000 132493
-24 113 0 6250000 133122
-23 116 0 6250000 133685
-22 118 0 6250000 134363
-21 117 0 6250000 134989
-20 118 0 6250000 135608
-19 121 0 6250000 136152
-18 123 0 6250000 136628
-17 125 0 6250000 137085
-16 126 0 6250000 137605
-15 127 0 6250000 138141
-14 128 0 6250000 138655
-13 131 0 6250000 139273
-12 131 0 6250000 139890
-11 133 0 6250000 140487
-10 134 0 625000

In [17]:
print('LR CPM: ',6210543/142114) 
print('LR CPC: ',6210543/(142*1000))

LR CPM:  43.701134300631885
LR CPC:  43.736218309859154


In [9]:
avgCTR = df_train['click'].sum()/df_train['click'].agg('count')
base = df_train['payprice'].sum()/df_train['payprice'].agg('count')
for i in range(-50,0):
    sum = 6250 * 1000
    clicks = 0
    payprices = 0
    num = 0
    for pay,click,predict in zip(df_val["payprice"],df_val['click'],y_pred_gbdt):
        sum_sub = sum - pay
        bidprice = (base * predict/avgCTR)-i
        if bidprice-pay > 0 and sum_sub >= 0:
            sum = sum_sub
            clicks += click
            payprices += pay
            num += 1
    print(i, clicks, sum, payprices, num)

-50 90 0 6250000 127576
-49 92 0 6250000 128289
-48 93 0 6250000 129206
-47 94 0 6250000 130138
-46 95 0 6250000 130803
-45 95 0 6250000 131632
-44 98 0 6250000 132774
-43 100 0 6250000 133686
-42 103 0 6250000 134700
-41 103 0 6250000 135506
-40 103 0 6250000 136557
-39 106 0 6250000 137720
-38 108 0 6250000 138759
-37 110 0 6250000 140032
-36 110 0 6250000 140894
-35 110 0 6250000 141824
-34 113 0 6250000 143186
-33 113 0 6250000 145063
-32 116 0 6250000 146205
-31 119 0 6250000 147267
-30 121 0 6250000 148744
-29 122 0 6250000 151019
-28 124 0 6250000 153006
-27 125 0 6250000 154250
-26 127 0 6250000 155592
-25 135 0 6250000 157340
-24 140 0 6250000 158846
-23 140 235165 6014835 155348
-22 140 382409 5867591 153036
-21 139 481937 5768063 151402
-20 138 665417 5584583 148453
-19 138 814158 5435842 146030
-18 134 994001 5255999 143004
-17 133 1128852 5121148 140699
-16 133 1272352 4977648 138293
-15 130 1442376 4807624 135491
-14 130 1581719 4668281 132930
-13 130 1798351 4451649 1291

In [11]:
print('GBDT CPM: ',6250000/158846) 
print('GBDT CPC: ',6250000/140000)



GBDT CPM:  39.34628508114778
GBDT CPC:  44.642857142857146


In [14]:
avgCTR = df_train['click'].sum()/df_train['click'].agg('count')
base = df_train['payprice'].sum()/df_train['payprice'].agg('count')
for i in range(-50,100):
    sum = 6250 * 1000
    clicks = 0
    payprices = 0
    num = 0
    for pay,click,predict in zip(df_val["payprice"],df_val['click'],y_pred_gbdtlr1):
        sum_sub = sum - pay
        bidprice = (base * predict/avgCTR)-i
        if bidprice-pay > 0 and sum_sub >= 0:
            sum = sum_sub
            clicks += click
            payprices += pay
            num += 1
    print(i, clicks, sum, payprices, num)

-50 110 0 6250000 131670
-49 111 0 6250000 132276
-48 112 0 6250000 133063
-47 115 0 6250000 133744
-46 117 0 6250000 134890
-45 121 0 6250000 135492
-44 121 0 6250000 136123
-43 120 0 6250000 136753
-42 120 0 6250000 137274
-41 122 0 6250000 137894
-40 125 0 6250000 138452
-39 127 0 6250000 139255
-38 129 0 6250000 140498
-37 130 0 6250000 141310
-36 134 0 6250000 142517
-35 136 0 6250000 143591
-34 136 0 6250000 144219
-33 137 0 6250000 144919
-32 138 0 6250000 145652
-31 139 0 6250000 146400
-30 141 0 6250000 147035
-29 147 0 6250000 148036
-28 152 0 6250000 149438
-27 155 20679 6229321 149804
-26 155 104162 6145838 148281
-25 153 309736 5940264 145105
-24 153 423031 5826969 142946
-23 151 531246 5718754 141051
-22 151 706285 5543715 138086
-21 150 799465 5450535 136244
-20 149 900429 5349571 134350
-19 148 1039637 5210363 131742
-18 147 1159379 5090621 129319
-17 147 1270957 4979043 126938
-16 147 1379414 4870586 124526
-15 145 1541960 4708040 121301
-14 145 1656749 4593251 118826


In [15]:
print('GBDT+LR CPM: ',6229321/149804) 
print('GBDT+LR CPC: ',6229321/155000)

GBDT+LR CPM:  41.583141972176975
GBDT+LR CPC:  40.189167741935485


In [18]:
import pandas as pd
filename = "test.csv"
columns = ('hour','weekday','useragent', 'slotprice', 'region', 'city', 'slotwidth', 'slotheight', 'advertiser', 'usertag')
df_test = pd.read_csv(filename,usecols=columns)
df_test.insert(1,'browser',df_test['useragent'])
browsers = []
for name in df_test["useragent"].map(lambda x:x.split('_')[1:][0]):
    if name not in browsers:
        browsers.append(name)
data_bro = []
for bro in df_test["useragent"].map(lambda x:x.split('_')[1:][0]):
    data_bro.append(browsers.index(bro))
df_test['browser'] = data_bro
    
df_test.insert(1,'os',df_test['useragent'])

oss = []
for name in df_test["useragent"].map(lambda x:x.split('_')[0]):
    if name not in oss:
        oss.append(name)
data_os = []
for os in df_test["useragent"].map(lambda x:x.split('_')[0]):
    data_os.append(oss.index(os))
df_test['os'] = data_os

df_test.insert(1,'size',df_test['slotwidth'])
width = df_test['slotwidth'].astype('str')
height = df_test['slotheight'].astype('str')
size = width.str.cat(height, sep='*')
df_test['size']=size


del df_test['slotwidth']
del df_test['slotheight']

df_test.insert(1,'floorprice',df_test['slotprice'])
floor = []
for i in df_test["slotprice"]:
    if i in range(1,10):
        floor.append(1)
    elif i in range(11,50):
        floor.append(2)
    elif i in range(51,100):
        floor.append(3)
    elif i == 0:
        floor.append(0)
    else:
        floor.append(4)
df_test['floorprice'] = floor 
col = ['slotprice','size', 'os', 'browser']
for column in col: 
    df_test[column]= pd.factorize(df_test[column].values , sort=True)[0] + 1
df_test['advertiser_n']= pd.factorize(df_test['advertiser'].values , sort=True)[0] + 1


import collections
users = collections.defaultdict()
i = 0
for user in df_test["usertag"]:
    users[i] = str(user).split(',')
    i += 1
s = pd.Series(users)
usrtag = pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)
del df_test["usertag"]

# dummy_price = pd.get_dummies(df_test['floorprice'], prefix='floorprice')  
# dummy_os = pd.get_dummies(df_test['os'], prefix='os')  
# dummy_browser = pd.get_dummies(df_test['browser'], prefix='browser')  
# dummy_weekday = pd.get_dummies(df_test['weekday'], prefix='weekday') 
# dummy_hour = pd.get_dummies(df_test['hour'], prefix='hour') 
# # cols_to_keep = ['click'] 
# data_test = df_test.join(dummy_price)
# data_test = data_test.join(dummy_os)
# data_test = data_test.join(dummy_browser)
# data_test = data_test.join(dummy_weekday)
# data_test = data_test.join(dummy_hour)
# data_test = data_test.join(usrtag)


data_test = df_test.join(usrtag)

In [19]:
del data_test['nan']
del data_test['useragent']

In [23]:
del data_test['advertiser']

In [24]:
X_t = data_test
X_t_lr = data_test

In [25]:
X_t.head()

,weekday,floorprice,size,os,browser,hour,region,city,slotprice,advertiser_n,...,13866,13874,14273,15398,16593,16617,16661,16706,16751,16753
0,0,4,13,1,1,12,146,159,8,8,...,0,0,0,0,0,0,0,0,0,0
1,3,1,16,2,2,14,1,1,3,5,...,0,0,0,0,0,0,0,0,0,0
2,5,0,17,1,3,19,27,34,1,1,...,1,0,0,0,0,0,0,0,0,0
3,0,2,27,1,3,21,238,245,28,4,...,0,0,0,0,0,0,0,0,0,0
4,2,4,13,1,3,20,27,35,48,7,...,0,0,0,0,0,0,0,0,0,0


In [26]:
X_train_leaves = gbdt.apply(X_train)[:,:,0]
X_test_leaves = gbdt.apply(X_t)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbdtenc = OneHotEncoder()
X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))



In [40]:
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train)

y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]



/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [27]:
import pandas as pd


filename = "train.csv"
df_train = pd.read_csv(filename)

filename = "test.csv"
data_test = pd.read_csv(filename)

avgCTR = df_train['click'].sum()/df_train['click'].agg('count')
base = df_train['payprice'].sum()/df_train['payprice'].agg('count')

In [28]:
# sum = 6250 * 1000
result_id = []
result_price = []
for bidid,predict in zip(data_test['bidid'],y_pred_gbdtlr1):
    bidprice = (base * predict/avgCTR)+27
    result_id.append(bidid)
    result_price.append(bidprice)
result=pd.DataFrame({'bidid':result_id,
                     'bidprice':result_price})
result.to_csv('result_click155.csv')